## 530. Нейрологотип

Цифровой художник готовится к своей первой профессиональной офлайн-выставке. Специально для её открытия он сделал блок с видеоартом. Художник часто использует нейросети в работе, и здесь он опять решил воспользоваться этим инструментом. Чтобы сгенерировать видеоарт, он использовал функцию генерации видео в Шедевруме. Художник сгенерировал десять вариантов видеофрагмента для своего произведения. Теперь ему предстоит определить лучший вариант. Он решил провести опрос в формате side-by-side: участники опроса просматривают пары видеороликов, и в каждой паре они должны выбрать один ролик, который больше понравится. Так как опрос проводился среди друзей и знакомых художника, анкет оказалось не очень много. Результаты представлены в файле в формате p1, p2, p1_votes, p2_votes (pi_votes — количество выборов видео i, когда была показана эта пара видеороликов). Художник обратился к вам за помощью: помогите проанализировать результаты опроса и определите топ-3 видеороликов, которые понравились участникам опроса, начиная с наилучшего. Hint: если топ-3 выделяется плохо, то попробуйте другой классический метод ранжирования для такого типа парных сравнений

### Формат ввода
Результаты представлены в файле в формате TSV (tab-separated values): https://disk.yandex.ru/d/fIjHfA-qOfysjQ
Название столбцов: p1, p2, p1_votes, p2_votes
pi - id логотипа (латинская заглавная буква от А до J)
pi_votes - кол-во выборов фрагмента i, когда была показана эта пара видеофрагментов

### Формат вывода
Введите в поле ответа ID трёх самых популярных видеофрагментов, начиная с наилучшего. Разделитель - пробел.

### Пример ответа: A B C

In [ ]:
import pandas as pd
data = pd.read_csv('dataset.tsv',sep = '\t')
data

,p1,p2,p1_votes,p2_votes
0,D,F,5,4
1,B,C,1,4
2,B,A,2,3
3,C,E,3,5
4,F,H,1,8
5,G,D,3,6
6,J,E,2,6
7,F,C,3,6
8,I,F,2,8
9,F,E,0,8


In [ ]:
data['best'] = None
data.loc[data['p1_votes'] > data['p2_votes'], 'best'] = data['p1']
data.loc[data['p1_votes'] < data['p2_votes'], 'best'] = data['p2']
data

,p1,p2,p1_votes,p2_votes,best
0,D,F,5,4,D
1,B,C,1,4,C
2,B,A,2,3,A
3,C,E,3,5,E
4,F,H,1,8,H
5,G,D,3,6,D
6,J,E,2,6,E
7,F,C,3,6,C
8,I,F,2,8,F
9,F,E,0,8,E


In [3]:
import pandas as pd

def rank_videos(tsv_file):
    """Ранжирует видеоролики на основе парных сравнений методом Elo.

    Args:
        tsv_file: Путь к TSV-файлу с результатами опроса.

    Returns:
        Список ID трех лучших видеороликов.
    """

    df = pd.read_csv(tsv_file, sep='\t')

    videos = sorted(set(df['p1']) | set(df['p2']))
    elo_ratings = {video: 1000 for video in videos}

    k_factor = 32 # Коэффициент Эло

    for _, row in df.iterrows():
        p1, p2, p1_votes, p2_votes = row
        r1 = elo_ratings[p1]
        r2 = elo_ratings[p2]

        e1 = 1 / (1 + 10*((r2 - r1) / 400))
        e2 = 1 / (1 + 10*((r1 - r2) / 400))

        s1 = p1_votes / (p1_votes + p2_votes)
        s2 = p2_votes / (p1_votes + p2_votes)
        
        elo_ratings[p1] = r1 + k_factor * (s1 - e1)
        elo_ratings[p2] = r2 + k_factor * (s2 - e2)


    sorted_videos = sorted(elo_ratings, key=elo_ratings.get, reverse=True)
    top_3 = sorted_videos[:3]

    return " ".join(top_3)



# Пример использования
tsv_file = 'dataset.tsv' # Замените на имя вашего TSV-файла
top_3_videos = rank_videos(tsv_file)
print(top_3_videos)


I J G
